In [4]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [5]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 1 PDF files to process

Processing: LLM24aug.pdf


Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 40 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 44 0 (offset 0)
Ignoring wrong pointing object 46 0 (offset 0)
Ignoring wrong pointing object 49 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)
Ignoring wrong pointing object 53 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 60 0 (offset 0)
Ignoring wrong pointing object 62 0 (offset 0)
Ignoring wrong 

  ✓ Loaded 72 pages

Total documents loaded: 72


In [6]:
all_pdf_documents

[Document(metadata={'producer': 'macOS Version 14.6.1 (Build 23G93) Quartz PDFContext', 'creator': 'PowerPoint', 'creationdate': "D:20240820140818Z00'00'", 'moddate': "D:20240820140818Z00'00'", 'source': '..\\data\\PDF\\LLM24aug.pdf', 'total_pages': 72, 'page': 0, 'page_label': '1', 'source_file': 'LLM24aug.pdf', 'file_type': 'pdf'}, page_content='Large Language ModelsIntroduction to Large Language Models'),
 Document(metadata={'producer': 'macOS Version 14.6.1 (Build 23G93) Quartz PDFContext', 'creator': 'PowerPoint', 'creationdate': "D:20240820140818Z00'00'", 'moddate': "D:20240820140818Z00'00'", 'source': '..\\data\\PDF\\LLM24aug.pdf', 'total_pages': 72, 'page': 1, 'page_label': '2', 'source_file': 'LLM24aug.pdf', 'file_type': 'pdf'}, page_content='Language models•Remember the simple n-gram language model•Assigns probabilities to sequences of words•Generate text by sampling possible next words•Is trained on counts computed from lots of text•Large language models are similar and diff

In [7]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [8]:
chunks=split_documents(all_pdf_documents)
chunks

Split 72 documents into 134 chunks

Example chunk:
Content: Large Language ModelsIntroduction to Large Language Models...
Metadata: {'producer': 'macOS Version 14.6.1 (Build 23G93) Quartz PDFContext', 'creator': 'PowerPoint', 'creationdate': "D:20240820140818Z00'00'", 'moddate': "D:20240820140818Z00'00'", 'source': '..\\data\\PDF\\LLM24aug.pdf', 'total_pages': 72, 'page': 0, 'page_label': '1', 'source_file': 'LLM24aug.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'macOS Version 14.6.1 (Build 23G93) Quartz PDFContext', 'creator': 'PowerPoint', 'creationdate': "D:20240820140818Z00'00'", 'moddate': "D:20240820140818Z00'00'", 'source': '..\\data\\PDF\\LLM24aug.pdf', 'total_pages': 72, 'page': 0, 'page_label': '1', 'source_file': 'LLM24aug.pdf', 'file_type': 'pdf'}, page_content='Large Language ModelsIntroduction to Large Language Models'),
 Document(metadata={'producer': 'macOS Version 14.6.1 (Build 23G93) Quartz PDFContext', 'creator': 'PowerPoint', 'creationdate': "D:20240820140818Z00'00'", 'moddate': "D:20240820140818Z00'00'", 'source': '..\\data\\PDF\\LLM24aug.pdf', 'total_pages': 72, 'page': 1, 'page_label': '2', 'source_file': 'LLM24aug.pdf', 'file_type': 'pdf'}, page_content='Language models•Remember the simple n-gram language model•Assigns probabilities to sequences of words•Generate text by sampling possible next words•Is trained on counts computed from lots of text•Large language models are similar and diff

embedding And vectorStoreDB

In [9]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


VectorStore

In [11]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [12]:
chunks

[Document(metadata={'producer': 'macOS Version 14.6.1 (Build 23G93) Quartz PDFContext', 'creator': 'PowerPoint', 'creationdate': "D:20240820140818Z00'00'", 'moddate': "D:20240820140818Z00'00'", 'source': '..\\data\\PDF\\LLM24aug.pdf', 'total_pages': 72, 'page': 0, 'page_label': '1', 'source_file': 'LLM24aug.pdf', 'file_type': 'pdf'}, page_content='Large Language ModelsIntroduction to Large Language Models'),
 Document(metadata={'producer': 'macOS Version 14.6.1 (Build 23G93) Quartz PDFContext', 'creator': 'PowerPoint', 'creationdate': "D:20240820140818Z00'00'", 'moddate': "D:20240820140818Z00'00'", 'source': '..\\data\\PDF\\LLM24aug.pdf', 'total_pages': 72, 'page': 1, 'page_label': '2', 'source_file': 'LLM24aug.pdf', 'file_type': 'pdf'}, page_content='Language models•Remember the simple n-gram language model•Assigns probabilities to sequences of words•Generate text by sampling possible next words•Is trained on counts computed from lots of text•Large language models are similar and diff

In [14]:
### Convert text to embeddings
texts=[doc.page_content for doc in chunks]

## generate Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store into the vectore database

vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 134 texts...


Batches: 100%|██████████| 5/5 [00:05<00:00,  1.19s/it]


Generated embeddings with shape: (134, 384)
Adding 134 documents to vector store...
Successfully added 134 documents to vector store
Total documents in collection: 134


Retiever pipeline From VectorStore

In [15]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [16]:
rag_retriever

In [17]:
rag_retriever.retrieve("What is LLM Models")

Retrieving documents for query: 'What is LLM Models'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  4.72it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


[{'id': 'doc_3d6c58d0_80',
  'content': 'Large Language ModelsPretraining data for LLMs',
  'metadata': {'page_label': '36',
   'page': 35,
   'creationdate': "D:20240820140818Z00'00'",
   'producer': 'macOS Version 14.6.1 (Build 23G93) Quartz PDFContext',
   'total_pages': 72,
   'source': '..\\data\\PDF\\LLM24aug.pdf',
   'doc_index': 80,
   'file_type': 'pdf',
   'content_length': 46,
   'creator': 'PowerPoint',
   'moddate': "D:20240820140818Z00'00'",
   'source_file': 'LLM24aug.pdf'},
  'similarity_score': 0.06880223751068115,
  'distance': 0.9311977624893188,
  'rank': 1},
 {'id': 'doc_7e0ffea5_89',
  'content': 'Large Language ModelsPretraining data for LLMs',
  'metadata': {'page_label': '43',
   'producer': 'macOS Version 14.6.1 (Build 23G93) Quartz PDFContext',
   'creationdate': "D:20240820140818Z00'00'",
   'content_length': 46,
   'doc_index': 89,
   'page': 42,
   'source': '..\\data\\PDF\\LLM24aug.pdf',
   'file_type': 'pdf',
   'source_file': 'LLM24aug.pdf',
   'moddate

Integration Vectordb context pipeline with LLM output

In [ ]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv

### Initialize the Groq LLM (set your GROQ_API_KEY in environment)
groq_api_key ="gYOUR API KEY"

llm=ChatGroq(groq_api_key=groq_api_key,model_name ="llama-3.1-8b-instant",temperature=0.1,max_tokens=1024)

## 2. Simple RAG function : retrive context +  generate response
def rag_simple(query,retriver,llm,top_k=3):
    ## retriver context
    results=retriver.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question"
    
    ## generate the answer using Groq LLM
    prompt=f"""Use the following context to answer the question concisely.
        context:
        {context}

        Question: {query}

        Answer:"""
        
    response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content

In [34]:
answer=rag_simple("What is encoders?",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'What is encoders?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 58.01it/s]

Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)


Encoders are neural networks that can condition on future context, allowing them to get bidirectional context.


Enchanced RAG Pipelin features

In [36]:
#----Enchanced RAG pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG Pipeline with extra features:
    - Returns answer, socurces, confidence score, and optionally full context.
    """
    
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return{'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context= "\n\n".join([doc['content'] for doc in results])
    sources =[{
        'source': doc ['metadata'].get('source_file', doc['metadata'].get('source','unknown')),
        'page' : doc ['metadata'].get('page','unkown'),
        'score' : doc['similarity_score'],
        'preview': doc ['content'][:120]+ '....'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate Prompt
    prompt= f"""Use the following context to answer the  question concisely.\ncontext:\n{context}\n\nQuestion: {query}\n\nAnswer: """
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer' : response.content,
        'sources' : sources,
        'confidence' : confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("What is LLM?", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'What is LLM?'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 31.55it/s]

Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)


Answer: Large Language Model (LLM)
Sources: [{'source': 'LLM24aug.pdf', 'page': 36, 'score': 0.12569493055343628, 'preview': 'LLMs are mainly trained on the webCommon crawl, snapshots of the entire web produced by the non- profit Common Crawl wit....'}]
Confidence: 0.12569493055343628
Context Preview: LLMs are mainly trained on the webCommon crawl, snapshots of the entire web produced by the non- profit Common Crawl with billions of pagesColossal Clean Crawled Corpus (C4; Raffel et al. 2020), 156 billion tokens of English,  filtered What's in it? Mostly patent text documents, Wikipedia, and news 


In [39]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("what is Temperature sampling?", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'what is Temperature sampling?'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 10.56it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)
Streaming answer:
Use the following context to answer the question concisely.
Context:
Temperature samplingReshape the distribution instead of truncating itIntuition from thermod

ynamics, •a system at high temperature is flexible and can explore many possible states,•a system at lower temperature is likely to explore a subset of lower energy (better) states. In low-temperature sampling,  (τ ≤ 1) we smoothly•increase the probability of the most probable words•decrease the probability of the rare words.

measure will be more robust in very different contexts, dynamically increasing and
decreasing the pool of word candidates.
Given a distribution P ( w
t
| w
< t
) , the top- p vocabulary V
( p )
is the smallest set of
words such that
X
w 2 V
( p )
P ( w | w
< t
)   p . (10.2)
10.2.3 Temperature sampling
In temperature sampling , we don’t truncate the distribution, but instead reshape
temperature
sampling
it. The intuition for temperature sampling comes from thermodynamics, where a
system at a high temperature is very ﬂexible and can explore many possible states,
while a system at a lower temperature is likely to explore a subset of lower energy
(better) states. In